In [1]:
!pip install fairseq
!pip install pandas torchaudio sentencepiece
!pip install datasets
!pip install wandb
!wandb login --relogin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 29.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11291641 sha256=8d21a28d805e5f588a92b2dfa20b67f986d32a62bbd4e8d59a9afb4d668d7671
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.wh

In [2]:
!git clone https://github.com/facebookresearch/fairseq.git

Cloning into 'fairseq'...
remote: Enumerating objects: 34887, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 34887 (delta 18), reused 33 (delta 17), pack-reused 34850
Receiving objects: 100% (34887/34887), 25.07 MiB | 14.37 MiB/s, done.
Resolving deltas: 100% (25308/25308), done.


In [3]:
%cd /content/fairseq

/content/fairseq


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
!mkdir /content/commonvoice

### Copy the final-commonvoice.py file into the /content/fairseq/examples/speech_to_text/ directory

In [8]:
%cd /content/fairseq/
!python examples/speech_to_text/final-commonvoice.py \
  --output-root /content/commonvoice/ --vocab-type unigram --vocab-size 2000

/content/fairseq
2023-11-03 00:16:44.039904: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 00:16:44.039960: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 00:16:44.039999: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 00:16:44.048472: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 00:16:45.557667: W ten

In [9]:
!fairseq-train /content/commonvoice/ --save-dir /content/commonvoice/save \
  --config-yaml config.yaml --train-subset train --valid-subset validated \
  --num-workers 4 --max-tokens 40000 --max-update 300000 \
  --task speech_to_text --criterion label_smoothed_cross_entropy --label-smoothing 0.1 --report-accuracy \
  --arch s2t_transformer_s --share-decoder-input-output-embed \
  --optimizer adam --lr 2e-3 --lr-scheduler inverse_sqrt --warmup-updates 10000 \
  --clip-norm 10.0 --seed 1 --update-freq 8 --max-epoch 50 --log-format tqdm \
  --wandb-project mnlp-a2-s1-50

2023-11-03 00:19:35.892343: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 00:19:35.892403: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 00:19:35.892447: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 00:19:35.901158: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 00:19:37.039482: W tensorflow/compiler/

In [15]:
LS_ROOT = '/content/commonvoice/'
SAVE_DIR = '/content/commonvoice/save'
CHECKPOINT_FILENAME = 'checkpoint_best.pt'
SUBSETS = ['test','validated']

# Loop over subsets
for SUBSET in SUBSETS:
    !fairseq-generate {LS_ROOT} --config-yaml config.yaml --gen-subset {SUBSET} \
    --task speech_to_text --path {SAVE_DIR}/{CHECKPOINT_FILENAME} \
    --max-tokens 50000 --beam 25 --scoring wer

2023-11-03 00:52:34.085364: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 00:52:34.085426: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 00:52:34.085467: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 00:52:34.094171: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 00:52:35.257465: W tensorflow/compiler/